<a href="https://colab.research.google.com/github/khushbooekhande/DrugRecommendationSystem/blob/master/Implementation/DiseasePrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import csv
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 
from sklearn.linear_model import LogisticRegression
import random

In [2]:
df = pd.read_csv('disease_symptom_dataset_unprocessed.csv')

In [3]:
df

,Disease,Count of Disease Occurrence,Symptom
0,UMLS:C0020538_hypertensive disease,3363.0,UMLS:C0008031_pain chest
1,NaN,NaN,UMLS:C0392680_shortness of breath
2,NaN,NaN,UMLS:C0012833_dizziness
3,NaN,NaN,UMLS:C0004093_asthenia
4,NaN,NaN,UMLS:C0085639_fall
...,...,...,...
1861,NaN,NaN,UMLS:C0425251_bedridden^UMLS:C0741453_bedridden
1862,NaN,NaN,UMLS:C0242453_prostatism
1863,UMLS:C0011127_decubitus ulcer,42.0,UMLS:C0232257_systolic murmur
1864,NaN,NaN,UMLS:C0871754_frail


In [4]:
data = df.fillna(method='ffill')
data.shape

(1866, 3)

In [5]:
# Clean Disease and Symptom Names
def clean_data(data):
    data_list = []
    data_name = data.replace('^','_').split('_')
    n = 1
    for names in data_name:
        if (n % 2 == 0):
            data_list.append(names)
        n += 1
    return data_list

In [6]:
disease_list = []
disease_symptom_dict = defaultdict(list)
disease_symptom_count = {}
count = 0

for idx, row in data.iterrows():
    
    # Get the Disease Names
    if (row['Disease'] !="\xc2\xa0") and (row['Disease'] != ""):
        disease = row['Disease']
        disease_list = clean_data(data=disease)
        count = row['Count of Disease  Occurrence']

    # Get the Symptoms Corresponding to Diseases
    if (row['Symptom'] !="\xc2\xa0") and (row['Symptom'] != ""):
        symptom = row['Symptom']
        symptom_list = clean_data(data=symptom)
        for d in disease_list:
            for s in symptom_list:
                disease_symptom_dict[d].append(s)
            disease_symptom_count[d] = count

In [7]:
# Writing the data in a csv file

with open("Symptoms_clean.csv","w") as csvfile:
    writer = csv.writer(csvfile)
    for key,values in disease_symptom_dict.items():
        for v in values:
            key = str.encode(key).decode('utf-8')
            writer.writerow([key,v,disease_symptom_count[key]])

In [8]:
columns = ['Disease', 'Symptoms', 'Count']
data_clean = pd.read_csv("Symptoms_clean.csv", names = columns, encoding ="ISO-8859-1")

In [9]:
data_clean

,Disease,Symptoms,Count
0,hypertensive disease,pain chest,3363.0
1,hypertensive disease,shortness of breath,3363.0
2,hypertensive disease,dizziness,3363.0
3,hypertensive disease,asthenia,3363.0
4,hypertensive disease,fall,3363.0
...,...,...,...
2125,affect labile,bedridden,45.0
2126,affect labile,prostatism,45.0
2127,decubitus ulcer,systolic murmur,42.0
2128,decubitus ulcer,frail,42.0


In [10]:
len(data_clean["Disease"].unique())

148

In [11]:
len(data_clean["Symptoms"].unique())

446

In [12]:
df = pd.DataFrame(data_clean)
df

,Disease,Symptoms,Count
0,hypertensive disease,pain chest,3363.0
1,hypertensive disease,shortness of breath,3363.0
2,hypertensive disease,dizziness,3363.0
3,hypertensive disease,asthenia,3363.0
4,hypertensive disease,fall,3363.0
...,...,...,...
2125,affect labile,bedridden,45.0
2126,affect labile,prostatism,45.0
2127,decubitus ulcer,systolic murmur,42.0
2128,decubitus ulcer,frail,42.0


# **METHOD 1**

In [13]:
df1=df.drop('Count',axis=1)
df1

,Disease,Symptoms
0,hypertensive disease,pain chest
1,hypertensive disease,shortness of breath
2,hypertensive disease,dizziness
3,hypertensive disease,asthenia
4,hypertensive disease,fall
...,...,...
2125,affect labile,bedridden
2126,affect labile,prostatism
2127,decubitus ulcer,systolic murmur
2128,decubitus ulcer,frail


In [14]:
'''df_dc = df[['Disease', 'Count']]
df_dc'''

"df_dc = df[['Disease', 'Count']]\ndf_dc"

In [15]:
df_s = df1['Disease']
df_s = pd.DataFrame(df_s)
df_s

,Disease
0,hypertensive disease
1,hypertensive disease
2,hypertensive disease
3,hypertensive disease
4,hypertensive disease
...,...
2125,affect labile
2126,affect labile
2127,decubitus ulcer
2128,decubitus ulcer


In [16]:
df_1 = pd.get_dummies(df.Symptoms)
df_1

,Heberden's node,Murphy's sign,Stahli's line,abdomen acute,abdominal bloating,abdominal tenderness,abnormal sensation,abnormally hard consistency,abnormally hard consistency,abortion,...,vomiting,weepiness,weight gain,welt,wheelchair bound,wheezing,withdraw,worry,yellow sputum,yellow sputum
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2125,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2126,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2127,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2128,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
df_pivoted = pd.concat([df_s,df_1], axis=1)
df_pivoted

,Disease,Heberden's node,Murphy's sign,Stahli's line,abdomen acute,abdominal bloating,abdominal tenderness,abnormal sensation,abnormally hard consistency,abnormally hard consistency,...,vomiting,weepiness,weight gain,welt,wheelchair bound,wheezing,withdraw,worry,yellow sputum,yellow sputum
0,hypertensive disease,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,hypertensive disease,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,hypertensive disease,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,hypertensive disease,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,hypertensive disease,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2125,affect labile,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2126,affect labile,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2127,decubitus ulcer,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2128,decubitus ulcer,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
df_pivoted = df_pivoted.groupby('Disease').sum()
df_pivoted

,Heberden's node,Murphy's sign,Stahli's line,abdomen acute,abdominal bloating,abdominal tenderness,abnormal sensation,abnormally hard consistency,abnormally hard consistency,abortion,...,vomiting,weepiness,weight gain,welt,wheelchair bound,wheezing,withdraw,worry,yellow sputum,yellow sputum
Disease,,,,,,,,,,,,,,,,,,,,,
Alzheimer's disease,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
HIV,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Pneumocystis carinii pneumonia,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
accident cerebrovascular,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
acquired immuno-deficiency syndrome,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
tonic-clonic seizures,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
transient ischemic attack,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tricuspid valve insufficiency,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [19]:
df_pivoted = df_pivoted.reset_index()

Disease prediction using the original dataset without considering the disease occurance count

In [20]:
cols = df_pivoted.columns
cols = cols[1:]
cols

Index(['Heberden's node', 'Murphy's sign', 'Stahli's line', 'abdomen acute',
       'abdominal bloating', 'abdominal tenderness', 'abnormal sensation',
       'abnormally  hard consistency', 'abnormally hard consistency',
       'abortion',
       ...
       'vomiting', 'weepiness', 'weight gain', 'welt', 'wheelchair bound',
       'wheezing', 'withdraw', 'worry', 'yellow  sputum', 'yellow sputum'],
      dtype='object', length=445)

In [21]:
X = df_pivoted[cols]
y = df_pivoted['Disease']

Multibinomial Naive bayes

In [22]:
mnb = MultinomialNB()
mnb = mnb.fit(X, y)
y_pred = mnb.predict(X)

In [23]:
pred_counts = 0
for i in range(0, len(y)):
  if y_pred[i]!= y[i]:
    pred_counts += 1
print("Total number of incorrect predictions:", pred_counts)
print("Accuracy of the MNB model in Predicting Disease: ", ((len(y)-pred_counts)/len(y))*100, '%')

Total number of incorrect predictions: 15
Accuracy of the MNB model in Predicting Disease:  89.86486486486487 %


Extra Tree Classifier

In [25]:
ext = ExtraTreesClassifier(n_estimators=100, random_state=0)
ext = ext.fit(X, y)
y_pred = ext.predict(X)

In [26]:
pred_counts = 0
for i in range(0, len(y)):
  if y_pred[i]!= y[i]:
    pred_counts += 1
print("Total number of incorrect predictions:", pred_counts)
print("Accuracy of the ExtraTree model in Predicting Disease: ", ((len(y)-pred_counts)/len(y))*100, '%')

Total number of incorrect predictions: 14
Accuracy of the ExtraTree model in Predicting Disease:  90.54054054054053 %


Decision tree classifier

In [27]:
for i in range(10, 111, 20):
  dtc = DecisionTreeClassifier(max_depth=i)
  dtc.fit(X,y)
  y_pred = dtc.predict(X)
  print("Accuracy of the DTree classifier model in Predicting Disease: ", accuracy_score(y, y_pred)*100, '%')

Accuracy of the DTree classifier model in Predicting Disease:  17.56756756756757 %
Accuracy of the DTree classifier model in Predicting Disease:  48.64864864864865 %
Accuracy of the DTree classifier model in Predicting Disease:  87.16216216216216 %
Accuracy of the DTree classifier model in Predicting Disease:  90.54054054054053 %
Accuracy of the DTree classifier model in Predicting Disease:  90.54054054054053 %
Accuracy of the DTree classifier model in Predicting Disease:  90.54054054054053 %


Support Vector Machine

In [28]:
svm = svm.SVC()
svm = svm.fit(X,y)
y_pred = svm.predict(X)
print("Accuracy of the SVM model in Predicting Disease: ", accuracy_score(y, y_pred)*100, '%')

Accuracy of the SVM model in Predicting Disease:  90.54054054054053 %


### Disease prediction using method 1

In [29]:
importances = dtc.feature_importances_
indices = np.argsort(importances)[::-1]

features = cols

In [30]:
feature_dict = {}
for i,f in enumerate(features):
    feature_dict[f] = i

Prediction using one symptom

In [31]:
feature_dict['shortness of breath']
symptomDR = [i/332 if i == 332 else i*0 for i in range(len(features))]
print(len(symptomDR))
symptomDR = np.array(symptomDR).reshape(1,len(symptomDR))

445


In [32]:
dtc.predict(symptomDR)

array(['kidney  disease'], dtype=object)

In [33]:
ext.predict(symptomDR)

array(['decubitus  ulcer'], dtype=object)

In [34]:
mnb.predict(symptomDR)

array(['hyperglycemia'], dtype='<U37')

In [35]:
svm.predict(symptomDR)

array(['decubitus  ulcer'], dtype=object)

Prediction using two symptoms

In [36]:
input_features = [feature_dict['pain chest'], feature_dict['shortness of breath']]
input_features
arr =[]
for i in range(len(features)):
  if i == input_features[0]:
    i = int(i/input_features[0])
  elif i == input_features[1]:
    i = int(i/input_features[1])
  else:
    i = 0
  arr.append(i)
arr = np.array(arr).reshape(-1,len(arr))

In [37]:
dtc.predict(arr)

array(['kidney  disease'], dtype=object)

In [38]:
ext.predict(arr)

array(['kidney  disease'], dtype=object)

In [39]:
mnb.predict(arr)

array(['kidney  disease'], dtype='<U37')

In [40]:
svm.predict(arr)

array(['kidney  disease'], dtype=object)

Prediction using three features

In [41]:
input_features = [feature_dict['pain chest'], feature_dict['shortness of breath'],  feature_dict['asthenia']]
input_features
arr =[]
for i in range(len(features)):
  if i == input_features[0]:
    i = int(i/input_features[0])
  elif i == input_features[1]:
    i = int(i/input_features[1])
  elif i == input_features[2]:
    i = int(i/input_features[2])
  else:
    i = 0
  arr.append(i)
arr = np.array(arr).reshape(-1,len(arr))

In [42]:
dtc.predict(arr)

array(['kidney  disease'], dtype=object)

In [43]:
ext.predict(arr)

array(['kidney  disease'], dtype=object)

In [44]:
mnb.predict(symptomDR)

array(['hyperglycemia'], dtype='<U37')

In [45]:
svm.predict(symptomDR)

array(['decubitus  ulcer'], dtype=object)

In [46]:
data_clean.head()

,Disease,Symptoms,Count
0,hypertensive disease,pain chest,3363.0
1,hypertensive disease,shortness of breath,3363.0
2,hypertensive disease,dizziness,3363.0
3,hypertensive disease,asthenia,3363.0
4,hypertensive disease,fall,3363.0


METHOD 2

In [47]:
df = pd.DataFrame(data_clean)

In [48]:
df_s = data_clean['Disease']

In [49]:
df_1 = pd.get_dummies(df.Symptoms)

In [50]:
df_pivoted = pd.concat([df_s,df_1], axis=1)

In [51]:
df_pivoted.drop_duplicates(keep='first',inplace=True)

In [52]:
df_pivoted[:5]

,Disease,Heberden's node,Murphy's sign,Stahli's line,abdomen acute,abdominal bloating,abdominal tenderness,abnormal sensation,abnormally hard consistency,abnormally hard consistency,...,vomiting,weepiness,weight gain,welt,wheelchair bound,wheezing,withdraw,worry,yellow sputum,yellow sputum
0,hypertensive disease,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,hypertensive disease,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,hypertensive disease,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,hypertensive disease,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,hypertensive disease,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
cols = df_pivoted.columns

In [54]:
cols = cols[1:]

In [55]:
print(df_pivoted['Disease'].value_counts().unique())

[28 25 24 23 22 21 20 19 18 17 16 15 14 13 12 11 10  9  8  7  6  3]


In [56]:
df_pivoted_1 = df_pivoted.groupby('Disease').head(3)
df_pivoted_2 = df_pivoted.groupby('Disease').head(5)
df_pivoted_3 = df_pivoted.groupby('Disease').head(7)
df_pivoted_4 = df_pivoted.groupby('Disease').head(10)
df_pivoted_5 = df_pivoted.groupby('Disease').head(12)
df_pivoted_6 = df_pivoted.groupby('Disease').head(15)
df_pivoted_7 = df_pivoted.groupby('Disease').head(17)
df_pivoted_8 = df_pivoted.groupby('Disease').head(20)
df_pivoted_9 = df_pivoted.groupby('Disease').head(24)
df_pivoted_10 = df_pivoted.groupby('Disease').head(28)

In [57]:
df_pivoted_1 = df_pivoted_1.groupby('Disease').sum().reset_index()
df_pivoted_2 = df_pivoted_2.groupby('Disease').sum().reset_index()
df_pivoted_3 = df_pivoted_3.groupby('Disease').sum().reset_index()
df_pivoted_4 = df_pivoted_4.groupby('Disease').sum().reset_index()
df_pivoted_5 = df_pivoted_5.groupby('Disease').sum().reset_index()
df_pivoted_6 = df_pivoted_6.groupby('Disease').sum().reset_index()
df_pivoted_7 = df_pivoted_7.groupby('Disease').sum().reset_index()
df_pivoted_8 = df_pivoted_8.groupby('Disease').sum().reset_index()
df_pivoted_9 = df_pivoted_9.groupby('Disease').sum().reset_index()
df_pivoted_10 = df_pivoted_10.groupby('Disease').sum().reset_index()

In [58]:
symptom_data = pd.concat([df_pivoted_1,df_pivoted_2,df_pivoted_3,df_pivoted_4,df_pivoted_5,
                      df_pivoted_6,df_pivoted_7,df_pivoted_8, df_pivoted_9, df_pivoted_10], axis=0, sort=True)

In [59]:
print(symptom_data['Disease'].value_counts())

primary  carcinoma of the liver cells    10
colitis                                  10
bipolar  disorder                        10
hypercholesterolemia                     10
oral  candidiasis                        10
                                         ..
glaucoma                                 10
cellulitis                               10
diverticulosis                           10
kidney  disease                          10
gastritis                                10
Name: Disease, Length: 148, dtype: int64


In [60]:
symptom_data = symptom_data.sort_values(by=['Disease'], ignore_index=True)

In [61]:
symptom_data = pd.concat([symptom_data]*20, ignore_index=True)

In [62]:
symptom_data = symptom_data.to_csv("symptoms_data.csv", index=False)
symptom_data = pd.read_csv("symptoms_data.csv")

In [63]:
symptom_data

,Disease,Heberden's node,Murphy's sign,Stahli's line,abdomen acute,abdominal bloating,abdominal tenderness,abnormal sensation,abnormally hard consistency,abnormally hard consistency,...,vomiting,weepiness,weight gain,welt,wheelchair bound,wheezing,withdraw,worry,yellow sputum,yellow sputum
0,Alzheimer's disease,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Alzheimer's disease,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Alzheimer's disease,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Alzheimer's disease,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,Alzheimer's disease,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29595,upper respiratory infection,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
29596,upper respiratory infection,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
29597,upper respiratory infection,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
29598,upper respiratory infection,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [64]:
x = symptom_data[cols]
y = symptom_data['Disease']

In [65]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [66]:
print ("DecisionTree")
dt = DecisionTreeClassifier(max_depth=120)
clf_dt=dt.fit(x_train,y_train)
print ("Acurracy: ", clf_dt.score(x_test,y_test))

DecisionTree
Acurracy:  0.888001638001638


In [67]:
mnb = MultinomialNB()
clf_mnb=mnb.fit(x_train,y_train)
print ("Acurracy: ", clf_mnb.score(x_test,y_test))

Acurracy:  0.8822686322686323


In [68]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(max_depth=12)
clf_rfc=rfc.fit(x_train,y_train)
print ("Acurracy: ", clf_rfc.score(x_test,y_test))

Acurracy:  0.8384520884520884


In [69]:
naivebys = GaussianNB()
clf_naivebys=naivebys.fit(x_train,y_train)
print ("Acurracy: ", clf_naivebys.score(x_test,y_test))

Acurracy:  0.891994266994267


In [70]:
from sklearn.svm import SVC
model = SVC()
clf_svm=model.fit(x_train,y_train)
print ("Acurracy: ", clf_svm.score(x_test,y_test))

Acurracy:  0.8894348894348895


In [71]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

importances = dt.feature_importances_
indices = np.argsort(importances)[::-1]

In [72]:
features = cols

In [73]:
feature_dict = {}
for i,f in enumerate(features):
    feature_dict[f] = i

In [74]:
input_features = [feature_dict['withdraw'], feature_dict['agitation'], feature_dict['hemiplegia']]  # These symptoms are of parkinson's disease
input_features

[441, 16, 177]

In [75]:
arr =[]
for i in range(len(features)):
  if i == input_features[0]:
    i = int(i/input_features[0])
  elif i == input_features[1]:
    i = int(i/input_features[1])
  elif i == input_features[2]:
    i = int(i/input_features[2])
  else:
    i = 0
  arr.append(i)

In [76]:
arr = np.array(arr).reshape(-1,len(arr))

In [77]:
dt.predict(arr)

array(['dementia'], dtype=object)

In [78]:
mnb.predict(arr)

array(['parkinson  disease'], dtype='<U37')

In [79]:
rfc.predict(arr)

array(['acquired  immuno-deficiency  syndrome'], dtype=object)

In [80]:
naivebys.predict(arr)

array(['parkinson  disease'], dtype='<U37')

In [81]:
model.predict(arr)

array(["Alzheimer's  disease"], dtype=object)